# Git
-----------------------------------

## Personalizing Git

```
git config --global user.name 'ntwo1980'
git config --global user.email 'ntwo1980@gmail.com'
git config --global color.ui auto
git config --global push.default simple
git config --global branch.autosetuprebase always
git config --global fetch.prune 1
git config --global log.abbrevCommit yes
git config --global core.excludesfile ~/.gitignore
git config --global core.abbrev 8
git config --global --add rerere.enabled 1
```

## Creating repository

### Creating a new, empty repository

```
$ git init
```

```
$ git init --bare
```

### Importing an existing project

```
$ git init
$ git add .
$ git commit -m 'init'
```

## Making commits

### Staging a new file

```
$ git add filename
```

Include all files in current index. This includes changed and deleted files, but not new ones.

```
$ git add -u
```

Include all filenames in the index and the working tree. This stages new files as well.

```
git add -A
```

### Adding partial changes

```
$ git add -p
```

### Removing a file

```
$ git rm filename
```

This does two things:
1. Deletes the file's entry from the index.
2. Deletes the working file as well.

### Deleting untracked files

```
$ git clean --force
```

To make `git clean` a bit safer, you can preview what will be removed.

```
git clean -n
```

### Deleting ignored files

```
$ git clean --force -X
```

The `-X` argument specifies that `git clean` should remove only ignored files from the working directory.

### Clean working directory

```
git reset --hard
git clean -xdf
```

### Renaming a file

```
$ git mv foo bar
```

### Unstaging changes

```
$ git rest
```

### Making a commit

```
$ git commit -m "commit message"
```

```
$ git commit -a
```

Adds all tracked, modified files to the index before committing. This commits changed and deleted files, but not new ones. It is equivalent to `git add -u` followed by `git commit`.

### Adding a single commit to the current branch

```
git cherry-pick 65ab35
```

### Assuming files are unchanged

```
git update-index --assume-unchanged ConnectString.config
```

List assume-unchanged files.

```
git ls-files -v | grep '^[hsmrck?]' | cut -c 3-
```

Stop assuming files are unchanged.

```
git update-index --no-assume-unchanged ConnectString.config
```

### A commit workflow

1. Use `git add` to stage a subset of your changes.
2. Run `git stash --keep-index`. This saves and undoes unstaged changes while preserving staged changed in the index, and resets working tree to match the index.
3. Examine working tree state to make sure selection of changes makes sense.
4. Run `git commit`.
5. Use `git stash pop` to restore remaining unstaged changes, and go back to step 1. Continue this process until all changes are committed, as confirmed by `git status` reporting "nothing to commit, working directory clean."

## Undoing and editing commits

### Changing the last commit

```
$ git commit --amend
```

It uses `git reset --soft HEAD^` and then runs `git commit --reedit-message` with the previous (now reset) commit as an argument.

### Discarding the last commit

```
$ git reset HEAD~
```

### Discarding any number of commits

```
$ git reset HEAD~3
```

When discarding more than one commit, some further options to `git reset` become useful:

```
--mixed
```

The default: makes the index match the given commit, but does not change the working files. Changes made since the last commit appear unstaged.

```
--soft
```

This resets the branch tip only, and does not change the index; the discarded commit's changes remain staged. You might use this to stage all the changes from several previous commits, and then reapply them as a single commit.

```
--merge
```

Tries to keep outstanding file changes while rewinding the branch: files with unstaged changes are kept, while files differing between HEAD nd the given commit are updated. If there is overlap between those sets, the reset fails.

```
--hard
```

Resets your working files to match the given commit, as well as the index. Any changes you've made since the discarded commit are permanently lost, so be careful with this option!

### Undoing a commit

```
$ git revert 9c6a1fad
```

### Partial undo

```
$ git revert -n 9c6a1fad
$ git reset
$ git add -p
$ git commit
$ git checkout .
```

The `-n` option to `git revert` tells Git to apply and stage the reverted changes, but stop short of making a commit. You then unstage all the changes with `git reset`, and restage only those you want using the interactive `git add -p`. Finally, after committing the subset of changes you want, you discard the reset by checking out the content of the index, overwriting the remaining applied changes from `git revert`.

### Editing a series of commits

```
$ git rebase -i HEAD~n
```

```
pick
```

Use the commit as-is. Git will not stop for this commit unless there is a conflict.

```
reword
```

Change just the commit message. Git allows you to edit the message before reapplying this commit.

```
edit
```

Change the commit contents (and message). Here, Git stops after remaking this commit and allows you to do whatever you want. The usual thing is to use `git commit --amend` to replace the commit, the `git rebase --continue` to let Git continue with the rebase operation. However, you could also insert further commits, perhaps splitting the original changes up into several smaller commits. Git simply picks up from where you leave off, with the next change you asked it to make.

```
squash
```

Make this commit's changes part of the preceding one. To meld several consecutive commits into one, leave the first one marked `pick` and mark the remaining ones with `squash`. Git concatenates all the commit messages for you to edit.

```
fixup
```

Like `squash`, but discard the message of this commit when composing the composite message.

At any point when Git stops, you can abort the entire process and return to your previous state with `git rebase --abort`.

## Branching

### Making a new branch

```
$ git checkout -b alvin
$ git checkout -b alvin 9c632fa
```

### Switching branches

```
$ git checkout commander
```

### Deleting a branch

```
$ git branch -d commander
```

```
$ git push origin :commander
```

Deleting the branch from the origin repository.

### Renaming a branch

```
$ git branch -m old new
```

Renaming a local branch.

```
$ git push -u origin new
$ git push origin :old
```

There is no direct way to rename the corresponding branch in a remote repository, you must separately push the new branch and delete the old one.

## Tracking other repositories

### Clone a repository

```
$ git clone http://nifty-software.org/foo.git
```

### Synchronization: Push and Pull

```
$ git push -u origin new-branch
```

Add local branch to remote and set up tracking for local branch in the usual way. After this initial setup you can use just `git push` on this branch to push to the same remote.

## Merge conflicts

```
$ git checkout chandra
$ git merge floyd
$ git diff
```

1. If you want to use Git's content-merging and conflict resolution machinery, but do not want to create a merge commit, use `git merge --squash`.
2. You can use `git merge -m` to specify a commit message.

### Resolving merge conflicts

1. `git log -p --merge`<`2`> shows all commits containing changes relevant to any unmerged files, on either branch, together with their diffs. This can help you identify the changes in the history that led to the conflicts.
2. If you want to discard all the changes from one side of the merge, use `git checkout --{ours,theirs} file`<`2`> to update the working file with the copy from the current or other branch, follow by `git add file`<`2`> to stage the change and mark the conflict as resolved.
3. Having done that, if you would like to apply some of the changes from the opposite side, use `git checkout -p branch file`. This starts an interactive loop that allows you to selectively apply or edit differing sections.

```
$ git checkout --ours foo.txt
$ git add foo.txt
$ git checkout -p floyd foo.txt
$ git add foo.txt
```

## Naming conmmits

### Naming individual commits

#### Ref Name

To find a ref named foo, Git looks for the following in order:

1. `foo`: Normally, these are refs used by Git internally, such as HEAD, MERGE_HEAD, FETCH_HEAD, and so on, and are represented as files directly under .git.
2. `refs/foo`.
3. `refs/tags/foo`: The namespace for tags.
4. `refs/heads/foo`: The namespace for local branches.
5. `refs/remotes/foo`: The namespace for remotes, though this would not ordinarily itself be a ref, but rather a directory containing the remote's ref.
6. `refs/remotes/foo/HEAD`: The default branch of the remote "foo".

Briefly, this means that `git checkout foo` will check out a tag named `foo` if there is one, otherwise, a branch; if there is neither, but there is a remote named foo, the it will check out the default branch of that remote.

### Names relative to a given commit

```
rev^n
```

For example, `master^2`; this refers to the nth parent of a commit, numbered starting at 1. Special cases:

* `rev^ = rev^1`
* `rev^0 = rev` if rev is a commit. If rev is a tag, then it is the commit to which the tag refers.

```
rev~n
```

For example, `HEAD~3`; this is the nth ancestor of rev, always following the first parent commit. Special cases:

* `rev~ = rev~1`
* `rev~0 = rev`

### Names relative to the reflog

Local branch names usually have a reflog: a log of commits that used to be the head of this branch, along with the actions that changed it each time: `commit`, `cherry-pick`, `reset`, and so on. You view the composite log with `git log -g`, which follows your trail from one branch log to another via checkouts. The syntax `refname@\{selector\}` allows you to name a single commit according to various criteria evaluated against your reflog:

```
refname@{time/date}
```

The time can be such expressions as:

* now
* yesterday
* last week
* 6 months ago
* two Saturdays past
* Sat Sep 8 02:09:07 2012 -0400(or meaningful subsets of this)
* 1966-12-06 04:33:00

You can use dots instead of spaces to avoid having to quote or escape spaces to the shell, to ease typing: `topic@{last.week}` instead of `topic@{"last week"}` or `topic@{last\ week}`.

```
refname@{n}
```

For nonegative n, this is the nth prior value of refname. Note that this need not be the same as `refname~n`, the nth prior commit on the branch.

You can omit refname to refer to the current branch (e.g., @{5}).

```
@{-n}
```

With a negative number, this is the current tip of the nth branch checked out before the current one.

### The upstream branch

The notation `foo@{upstream}` (or just `foo@{u}`) names the branch upstream of the branch `foo`, as defined by the repository configuration. It just gives the object ID of the upstream branch head, though; options to `git rev-parse` are useful to find out the upstream branch name:

```
$ git rev-parse HEAD@{upstream}
b801f8bf1a76ea5c6c6ac7addee2bc7161a79c93

$ git rev-parse --abbrev-ref HEAD@{upstream}
origin/master

$ git rev-parse --symbolic-full-name-ref HEAD@{upstream}
refs/remotes/origin/master
```

### Matching a commit message

```
rev^regexp
```

For example, `HEAD^{/"fixed pr#1234"}`; this selects the youngest commit reachable from rev whose commit message matches the given regular expression.

## Naming sets of commits

* master, the commits on the master branch.
* master..topic, the commits on topic not yet merged into master.
* master...topic, the commits by which the topic and master branches differ.

```
$ git rev-list rev | git name-rev --stdin --name-only
```

Print out the commit set using names relative to local branches and tags.

## Viewing history

### Command format

```
$ git log [options] [commits] [[--] path ...]
```

### Output formats

```
$ git log --format=oneline
$ git log --format=medium
```

### Custom output format

```
git log --format="%ar %an did: %s"
```

### Limiting commits to be shown

```
-n (-n n, --max-count=n)
```

Only show the first n commits.

```
--skip=n
```

Skip n leading commits before starting output.

```
--{before,after}=date
```

Show commits made before or after a specific date.

```
--{author,committer}=regexp
```

Show only commits whose author or committer header (name <email>) matches the given regular expression.

```
--grep=regexp
```

Show only commits whose log messages match the given regular expression. Use `--grep-reglog` to match reflog entries instead, when using `git log -g` to examine the reflog instead of the commit graph.

```
--{min,max}-patterns=n
```

Show only commits with a matching number of parent commits. Synonyms:

* `--merges = --min-patterns=2`
* `--no-merges = --max-parents=1`

```
--first-parent
```

Follow only the first parent of a merge commit, rather than all of them. This shows only the activity on the topic branch itself, rather than commits brought in from the main branch by merging.

```
--diff-filter=[A|C|D|M|R|T]
```

Show commits containing files with any of the statuses given by the following one-letter codes.

* A: Added
* C: Copied
* D: Deleted
* M: Modified
* R: Renamed
* T: Type changed (e.g., a file replaced by a symbolic link)

### Regular Expressions

A number of options affect the interpretation of regular expressions:

```
-i (--regexp-ignore-case)
```

Ignore case differences.

```
-E (--extended-regexp)
```

Use extended regular expressions; the default type is basic.

```
-F (--fixed-strings)
```

Consider the limiting patterns as literal strings to be matched.

```
--perl-regexp
```

Use Perl-style regular expressions.

### Showing who last changed each line of a file

```
git blame --date=short file.cs
```

### Finding which commit caused a particular bug

```
git bisect start
git bisect bad
git bisect good 6576b6
git bisect run ls file.cs
```

### Reflog

`git log --walk-reflog (-g)` shows a completely different log: the *reflog*.

### Decoration

`git log --decorate={no,short,full}` shows refs pointing to the listed commits.

```
git log --decorate
```

### Date Style

```
git log --date={local,relative,default,iso,rfc,short,raw}
```

### Listing changed files

`git log --name-status` summarizes which files changed in a given commit (relative to its predecessor), and the nature of the changes.

`git log --name-only` lists only filenames without the status codes, and the `stat` gives an ASCII-art graph ("diffstat") representing the amount the amount and kind of changes in each file.

`git log --dirstat` summarizes the amount of change in subdirectories.

### Showing and following renames or copies

To enable renaming detection, use `--find-renames[=n]`(-M[n]). The optional integer *n* is an index of similarity (the default is 100%).

```
$ git log --name-status -M
```

To have Git follow a file past a rename, use `git log --follow`; this only works when you give a single file to follow:

```
$ git log --follow bar
```

`git log --find-copies[=n]` (-C[n]) does the same for detecting copies as `-M` does for renames. `-CC` (or `--find-copies-harder`) will consider all files in a commit as potential sources of copying, while plain `-C` considers only files that changed in that commit.

### Searching for changes: the "pickaxe"

The Git "pickaxe", `git log -S string`, lists commits that changed the number of occurrences of string in at least one file. `git log -G pattern` does the same with a regular expression.

### Showing diffs

`git log -p` shows the "patch" or "diff" associated with each commit. Normally, no diff is shown for merge commits, however you can use these options:

`-m`

Shows each pairwise diff between the merge and its parents.

`-c`

Shows the differences with all parents simultaneously in a merged format, rather than serially as with `-m`, and only for files that were modified in all branches.

`--cc`

Implies `-c` and further implifies the diff by showing only conflicts; change regions with only two variants of which the merge picked one unmodified are not shown.

#### Color

The option `--color[={always,auto,never}]` uses color to help distinguish difference regions.

#### Word diff

The option `--word-diff[={plain,color,none}]` shows word level changes within lines, rather than entire changed lines.

### Comparing branches

```
$ git log --cherry-pick master...other
```

`git log --cherry-pick` omits commits that have identical diffs, that is cherry-pick commit.

```
$ git log --cherry-mark master...other
```

The variation `--cherry-mark` will mark duplicate commits with an equal sign, instead of omitting them.

#### Displaying sides

```
$ git log --cherry-pick --right-only master...other
```

This shows commits on *other* that are not contained in *master* or patch-equivalent to another commit in their difference. And similar to `--cherry-mark`, the related option `--left-right` displays the side of a commit with the symbols < and >:

```
$ git log --cherry-mark -left-right master...other
```

The simple option `--cherry` is a synonym for `--right-only --cherry-mark --no-merges`, so that:

```
$ git log --cherry HEAD@{upstream}...
```

Shows the commits on your side of the current branch (ignoring possible merges with other branches), marking those the duplicate changes made by distinct commits on the other side.

### Commit ordering

Normally, `git log` displays commits in reverse chronological order according to the committer (not author) timestamps. You can alter this in three ways:

* `--date-order` shows all children before their parents;
* `--topo-order` implies `--date-order`, and also groups commits from the same branch together.
* `--reverse` reverses the output list.

### Related commands

#### git cherry

```
git cherry [-v] [upstream [head [limit]]]
```

This command is similar to `git log --cherry`, but more specialized. It shows commits on a branch that are not in the upstream, marking those whose changes are duplicated by distinct upstream commits with a minus sign (while other commits have a plus sign).

```
$ git cherry -v --abbrev
```

#### git shortlog

`git shortlog` summarizes commit history, grouping commits by author with the number of commits and their subjects, and applying a mailmap if available to rewrite author names or email addresses.

## Editing history

### Rebasing

```
$ git rebase [--onto newbase] [upstream] [branch]
```

```
$ git rebase
```

Actually means:

```
$ git rebase --onto origin/master origin/master master
```

#### Undoing a rebase

```
$ git log -g
$ git reset --hard e3a1d5b0
```

### Importing from one repository to another

#### Importing disconnected history
#### Importing linear history
#### Importing nonlinear history

### Commit surgery: git replace

### The big hammer: git filter-branch

```
git filter-branch --prune-empty --index-filter "git rm --cached --ignore-unmatch file.cs" master
```

## Reset, Checkout and Revert

`git reset` modifies the current branch pointer so it points to another commit. `git checkout` modifies the HEAD point so it points to another branch (or,rarely, commit). If you're on the master branch, `git reset --hard v0.1-release` sets the master branch to point to the top of the v0.1-release branch, whereas `git checkout v0.1-release` changes the current branch (the HEAD pointer) to point to the v0.1-release branch.

     command       | Scope        | Common use cases
------------------ | ------------ | -----------------
git reset          | Commit-level | Discard commits in a private branch or throw away uncommitted changes
git reset          | File-level   | Unstage a file
git checkout       | Commit-level | Switch between branches or inspect old snapshots
git checkout       | File-level   | Discard changes in the working directory
git revert         | Commit-level | Undo commits in a public branch
git revert         | File-level   | (N/A)